## Access caching data

In [ ]:
%matplotlib inline
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from datetime import datetime
import math
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import gridspec
from datetime import datetime
import numpy as np
import pandas as pd
import networkx as nx

es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60)

## Get the data from Elasticsearch, and store in csv

In [ ]:
my_query = {
    'query':{
        'bool':{
            'must':[
                {
                    'range': {
                        'accesses': {"gt":-1}
                    }
                }
            ]               
        }
    }
}

scroll = list(scan(client=es, index="caching-v1", query=my_query))


In [ ]:
count=0
accesses=[]
fn=[]
scope=[]
totalBuckets=[]
cachedBuckets=[]
for r in scroll:
    if count<2: print(r)
    count+=1
    accesses.append(r['_source']['accesses'])
    f=r['_source']['file'].replace('/cache/localroot/atlas/rucio/','').replace('.cinfo','')
    [sc,filen]=f.split(":")
    fn.append(filen)
    scope.append(sc)
    totalBuckets.append(r['_source']['full'])
    cachedBuckets.append(r['_source']['cached'])

a=pd.DataFrame({"accesses":accesses, "scope":scope, "fn": fn, "totalBuckets": totalBuckets, "cachedBuckets":cachedBuckets})
print(count)

### saving in pickle files

In [ ]:
a.to_csv("Data/CachingData.csv")
